In [ ]:
from z3 import *
import re

def clean_z3_code(code_block):
    """
    Clean the Z3 code block by removing unnecessary syntax and comments.
    """
    # Remove 'smt2' if present
    code_block = re.sub(r'^smt2\s*', '', code_block)
    
    # Remove any trailing whitespace
    lines = [line.strip() for line in code_block.split('\n')]
    
    # Filter out empty lines and join back
    return '\n'.join(line for line in lines if line)

def execute_z3_code(code_block):
    """
    Execute a Z3 code block and return the solver results.
    """
    # Clean the code block first
    code_block = clean_z3_code(code_block)
    
    # Create a complete namespace for Z3 execution
    namespace = {
        # Sorts
        'StringSort': StringSort,
        'IntSort': IntSort,
        'BoolSort': BoolSort,
        'RealSort': RealSort,
        'ArraySort': ArraySort,
        
        # Functions and operations
        'Function': Function,
        'StringVal': StringVal,
        'IntVal': IntVal,
        'RealVal': RealVal,
        'BoolVal': BoolVal,
        'Not': Not,
        'And': And,
        'Or': Or,
        'Implies': Implies,
        
        # Additional Z3 utilities
        'Solver': Solver,
        'sat': sat,
        'unsat': unsat,
        'unknown': unknown
    }
    
    try:
        # Execute the code block to get predicates, constants, axioms, and query
        exec(code_block, namespace)
        
        # Get axioms and query from the namespace
        axioms = namespace.get('axioms', [])
        query = namespace.get('query')
        
        if not axioms or query is None:
            return None, None, "Invalid code block: missing axioms or query"
        
        # Create solver and add axioms
        solver = Solver()
        for axiom in axioms:
            solver.add(axiom)
        
        # Add negation of query
        solver.add(Not(query))
        
        # Check satisfiability
        result = solver.check()
        
        if result == unsat:
            return axioms, query, "Query is true"
        elif result == sat:
            model = solver.model()
            return axioms, query, "Query is false"
        else:
            return axioms, query, "Query is unknown"
        
    except Exception as e:
        return None, None, f"Error executing code: {str(e)}"

def test_z3_code(iteration, right_answer, hallucinated_answer, COUNT):
    """
    Test both right and hallucinated answers and print results.
    """
    print(f"\nProcessing Iteration {iteration}")
    print("="*50)
    
    #print("\nTesting Right Answer:")
    #print("-"*30)
#     try:
#         axioms, query, result = execute_z3_code(right_answer)
#         if axioms is not None and query is not None:
#             print("Axioms:")
#             for axiom in axioms:
#                 print(f"  {axiom}")
#             print(f"Query: {query}")
#         print(f"Result: {result}")
#     except Exception as e:
#         print(f"Error: {str(e)}")
    
    print("\nTesting Hallucinated Answer:")
    print("-"*30)
    try:
        axioms, query, result = execute_z3_code(hallucinated_answer)
        if axioms is not None and query is not None:
            print("Axioms:")
            for axiom in axioms:
                print(f"  {axiom}")
            print(f"Query: {query}")
        print(f"Result: {result}")

    except Exception as e:
        print(f"Error: {str(e)}")
        
    return result
        

COUNT = 0
# Read the formatted snippets file
with open('formatted_snippet_2.txt', 'r') as f:
    content = f.read()

# Extract code blocks
iterations = extract_code_blocks(content)

# Process each iteration
for iteration, right_answer, hallucinated_answer in iterations:
    result = test_z3_code(iteration, right_answer, hallucinated_answer, COUNT)
    if result == "Query is false":
        COUNT += 1
    print("TOTAL CORRECT: ", str(COUNT))

